In [2]:
import pandas as pd

X_train = pd.read_excel('../data/x_train_data.xlsx')
X_val = pd.read_excel('../data/x_val_data.xlsx')
X_test = pd.read_excel('../data/x_test_data.xlsx')

y_train=pd.read_excel('../data/y_train_rent.xlsx')
y_val=pd.read_excel('../data/y_val_rent.xlsx')
y_test=pd.read_excel('../data/y_test_rent.xlsx')

In [4]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=42)

In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

MAE: 469.060945084399
RMSE: 1404991.3041467154
R² Score: 0.6518280273205725


In [9]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Προβλέψεις και αξιολόγηση
y_pred = model.predict(X_test)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", root_mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

MAE: 478.3153991699219
RMSE: 1197.146240234375
R² Score: 0.6448477506637573


In [11]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Ορίζουμε custom scorer για RMSE
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Ορίζουμε το μοντέλο
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Grid των υπερπαραμέτρων
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 1.0]
}

# Grid Search με 5-fold CV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring=rmse_scorer,
    cv=5,
    verbose=1,
    n_jobs=-1
)

# Εκπαίδευση
grid_search.fit(X_train, y_train)

# Καλύτερα αποτελέσματα
print("Best parameters:", grid_search.best_params_)
print("Best RMSE score (CV):", grid_search.best_score_)

# Αξιολόγηση στο test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Test RMSE:", mean_squared_error(y_test, y_pred))


Fitting 5 folds for each of 36 candidates, totalling 180 fits


c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
Best RMSE score (CV): nan
Test RMSE: 2030467.25


In [12]:
print(X_train.isnull().sum().sum())  # Αν χρησιμοποιείς DataFrame
print(np.isnan(X_train).sum())
print(np.isnan(y_train).sum())


0
res_date                          0
res_sqr                           0
construction_year                 0
levels                            0
bedrooms                          0
                                 ..
location_name_Φιλοθέη - Ψυχικό    0
location_name_Φυλή                0
location_name_Χαλάνδρι            0
location_name_Χαϊδάρι             0
location_name_Χολαργός            0
Length: 91, dtype: int64
rent_price    0
dtype: int64


In [14]:
from sklearn.metrics import mean_squared_error, make_scorer

# Η sklearn χρησιμοποιεί scoring όπου μεγαλύτερο είναι καλύτερο,
# άρα βάζουμε αρνητικό MSE
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring=rmse_scorer,
    cv=5,
    verbose=1,
    n_jobs=-1
)
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.7, 1.0]
}
print(grid_search)


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
   

In [16]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
import numpy as np

# Ορισμός scorer για RMSE (αρνητικό για GridSearchCV)
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

# Ορισμός μοντέλου
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Grid υπερπαραμέτρων
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.7, 1.0]
}

# Ορισμός GridSearchCV με 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring=rmse_scorer,
    cv=5,
    verbose=1,
    n_jobs=-1
)

# Εκπαίδευση του Grid Search
grid_search.fit(X_train, y_train)

# Εκτύπωση καλύτερων παραμέτρων και βαθμολογίας (CV)
print("Best parameters found:", grid_search.best_params_)
print("Best RMSE score (CV):", -grid_search.best_score_)  # αρνητικό επειδή ήταν make_scorer

# Αξιολόγηση στο test set με το καλύτερο μοντέλο
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Test MAE:", mean_absolute_error(y_test, y_pred))
print("Test RMSE:", mean_squared_error(y_test, y_pred))
print("Test R² Score:", r2_score(y_test, y_pred))


Fitting 5 folds for each of 16 candidates, totalling 80 fits


c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best parameters found: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.7}
Best RMSE score (CV): nan
Test MAE: 534.8925170898438
Test RMSE: 1426499.375
Test R² Score: 0.6464980840682983
